# Project 7: Random Forest

![](https://ars.els-cdn.com/content/image/1-s2.0-S0031320310003973-gr1.jpg)

## Instructions

### Description

In this project, we are going to explore Random Forest Regressors for predicting the weather, specifically the temperature. You'll load the data in, set up a Random Forest, then test its performance on both the original data set and new data. This new data will be the current weather conditions and will come from an Application Programming Interface(API).

### Grading

For grading purposes, we will clear all outputs from all your cells and then run them all from the top.  Please test your notebook in the same fashion before turning it in.

### Submitting Your Solution

To submit your notebook, first clear all the cells (this won't matter too much this time, but for larger data sets in the future, it will make the file smaller).  Then use the File->Download As->Notebook to obtain the notebook file.  Finally, submit the notebook file on Canvas.

## Side note:
Most of the tasks below require you use specific variable names. This is so that the cells with "assert" statements in them can check if you did the tasks correctly.

In [ ]:
# this might be necessary
#!pip3 install pprint

In [ ]:
import json
import pydot
import requests
import graphviz
import numpy as np
import pandas as pd
from pprint import pprint
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score

## Loading the data
You're given a csv file, ```temps.csv```, with weather data in Seattle from 2016.

## Tasks:
- Read in the file into a dataframe called `data`
- Run `get_dummies()` on the day of the week
- Specify the predictors (features) and targets:
 - Predictors: 'temp_1', 'temp_2', 'average', and the days of the week
 - Target: 'actual' (the actual temperature on the specified date)
- Split the data into training and testing sets

In [ ]:
# read csv
data = pd.DataFrame(pd.read_csv("temps.csv"))

# get_dummies
data = pd.get_dummies(data,columns= ['week'])

# train_test_split
features = ['temp_1','temp_2','week_Fri','week_Mon','week_Sat','week_Sun','week_Thurs','week_Tues','week_Wed']
target = ['actual']
X_train,X_test,Y_train,Y_test = train_test_split(data[features],data[target])

In [ ]:
for column in ['week_Fri', 'week_Mon', 'week_Sat', 'week_Thurs', 'week_Sun', 'week_Tues', 'week_Wed']:
    assert column in data.columns

## Random Forest 

### Background
Random Forests, aka Random Decision Trees, are an example of "ensemble: learning methods. They operate by constructing many decision trees at once and outputting their mean prediction. This is useful because decision trees on their own are prone to overfitting. Random Forests are most often used for classification and regression, but today, we'll only be doing regression.

### Tasks 
- Initialize a `RandomForestRegressor` with 1000 estimators and a max_depth of 10. The model will perform well with these values, but you can always change them later. Store the regressor in the variable `rf_clf`.
 - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html 
- Fit the model with `rf_clf.fit()`

In [ ]:
## initialize model
rf_clf = RandomForestRegressor(n_estimators = 1000,max_depth=10)

In [ ]:
## training
rf_clf.fit(X_train,Y_train.values.ravel())

## Tree Visualization

The next cell plots an image of one of the trees in the forest. It also  
saves a png image file in the same directory as this jupyter notebook file.

In [ ]:
# Plot a decision tree
index = 5
tree = rf_clf.estimators_[index]
from IPython.display import display
columns = list(X_train.columns)
#display(graphviz.Source(export_graphviz(tree, feature_names=columns, class_names=True, out_file=None)))

# Save image to png file
#export_graphviz(tree, out_file = 'tree.dot', feature_names = features, rounded = True, precision = 1)
#(graph, ) = pydot.graph_from_dot_file('tree.dot')
#graph.write_png('tree.png')

## Importance Visualization

Plot the learned importance of the features

In [ ]:
# Plot importances
importances = list(rf_clf.feature_importances_)
labeled_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
indices = range(len(importances))
plt.bar(indices, importances, orientation = 'vertical')
plt.title('feature importances')
plt.xticks(indices, features, rotation='vertical')
plt.show()

## Testing
First, we are going to use the test set to make predictions, same as always. Then we're going to get new data from an API.

### Tasks
- Find the R^2 value. Do this with `model.score()` or `sklearn.metrics.r2_score()` and save to a variable called `r2`. 
- Find the explained variance with `sklearn.metrics.explained_variance_score()` and save to a variable called `ev`.

For both of these metrics, the closer to 1, the better.

In [ ]:
import sklearn
r2 = rf_clf.score(X_test,Y_test)
#print(r2)
ev = sklearn.metrics.explained_variance_score(Y_test,rf_clf.predict(X_test))
#print(ev)

In [ ]:
assert r2 > 0.7
assert ev > 0.7

### API
#### Background
An application programming interface (API) is just another way to gather data by letting clients (us and our computers) programmatically interact with servers. Today, we're going to be using a weather API for getting the current temp of a location.

##### JSON
JavaScript Object Notation is a very popular data format, especially for APIs. It appears very similar to Python's dictionaries, with curly braces to start, then key-value pairs of data. An example for a person could be:
```
{
    'firstname': 'John',
    'lastname': 'Doe',
    'age': 55,
    'retired': true,
    'addresses': ['123 street', '456 street']
}
```
When we make an API request, the server will handle it, then respond with a bunch of JSON, which we can grab exactly what we want from.

#### OpenWeatherMap API
Let's use the OpenWeatherMap API to get the current weather. The documentation is [here](https://openweathermap.org/current). To do so, we use the `requests` library. Making a request is very similar to going to a website in a browser -- a URL points to a location or resource on a server and that server fulfills the tasks we ask it to.
<br>
The URL is `'http://api.openweathermap.org/data/2.5/weather?q={city name},{country code}'`, where we fill in the city name and country code(without the curly braces).
<br>
An example response the API would send: https://samples.openweathermap.org/data/2.5/weather?q=London,uk&appid=b6907d289e10d714a6e88b30761fae22


#### Your tasks
- Fill in the URL with the city and country_code. Set this to variable `url`.
- Use `requests.get()` to make request. Set the output to variable `r`. This takes two arguments: one called `url`, which is the URL above, and one called `params`, which is a dictionary of parameters, which in this case (and with most APIs) is a unique key that authenticates us and is an indication to use imperial units instead of Kelvin.
- Read the response with `r.text`
- Parse the response into JSON with `json.loads(response)`
- If you want, print the JSON out, formatted nicely with `pprint(response)`
- Get the current temperature from the JSON like you would a python dictionary. For example, `person['age']` if we were using the example JSON above. Set this value to the variable `current_temp`
- Compare the current temperature to the predicted value.

In [ ]:
city = 'seattle'
country_code = 'us'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city},{country_code}'
parameters = {
    'APPID' : '2dbcde0477e10f32f587960671d2f32e',
    'units' : 'imperial'
}

response = r.text
response=json.loads(response)
current_temp = response['main']['temp']
try:
    r = requests.get(url,parameters)   
except Exception as e:
   print(e)
#print(current_temp)

In [ ]:
assert response
assert current_temp

Now we want to compare current weather data to last year. The code given below gets the day of the year it is today, extracts that row from our original csv data, and splits that row into features and targets.

#### Your task
- Use the your random forest model to predict the temperature on this day last year and compare that prediction to the current temperature.
- Divide that comparision (the difference) by the current temperature to determine what percent error we have. Save this to the variable `error`.

In [ ]:
day_of_year = datetime.now().timetuple().tm_yday
last_year_features = np.array([data.loc[day_of_year, features]])
last_year_temp = data.at[day_of_year, 'actual']

predicted_temp = rf_clf.predict(last_year_features)
error = (predicted_temp-current_temp)/(current_temp)

In [ ]:
assert error

**Note**: This isn't really the error, since we're compaing the same day in different years. This is due to the limitations of the free tier of this API, but we can still get a *decent* idea of how well the model faired.

## Reflection Questions

Save your response to the questions below in the variable `response`, in the next cell.
- Do you think the model did "well"? Why or why not?
- Whether or not the model did well, what are some ways we could make it even better?
- What are some limitations to this model? Could we avoid/prevent them?

In [ ]:
response = 'I think that the model did well because the explained variance was close to one and when testing the predicted temp was also close to the real temp. It could be better if dimensionality reduction was applied to it. Some limitaions of this model are that it is randomized so it will never be exact and this cannot be prevented.'

In [ ]:
assert len(response) > 50